# 1. Sequence to Sequence Learning with Neural Networks

- Pytorch와 torchtext를 사용하여 sequence에서 sequence로 이동하는 NMT 모델을 구축하자!
- many to many를 푸는 모든 문제에 적용할 수 있는 구조에요

## Introduction
- 일반적인 Seq2Seq은 RNN을 사용하여 입력 문장을 단일 벡터로 인코딩하는 Enc-Dec 모델이에요
- 위 단일 벡터를 `context vecor`라고 부를게요
- context vector는 전체 입력 문장의 추상적인 표현이에요
- 이 벡터는 한 번에 한 단어씩 생성하여 대상 문장을 출력하는 방법을 학습하는 Decoder RNN에 의해 Decode되서 타켓 문장을 만들어내요

![img](https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013/assets/seq2seq1.png)

- 위 이미지는 번역 예시를 보여주고 있어요
- guten morgen, 좋은 아침! 이라는 뜻이죠?
- 위 문장은 시작 토큰, guten, morgen, 종료 토큰으로 분절(tokenize)되어 Encoder의 input으로 들어가게 되요
- 각 time-step별 input $x_t$와 이전 time-step의 hidden state $h_{t-1}$이 각 RNN Cell에서 연산되서 다음 hidden state를 구축하게 되요
- 아래와 같이 수식을 쓸 수 있겠군요!

$$h_t = EncoderRNN(e(x_t),h_{t-1})$$

- RNN의 vanishing gradient, exploding gradient 문제 등으로 해당 Cell은 LSTM 혹은 GRU를 사용해요!
- 마지막 time step `<eos>`과 그 이전 시점의 hidden state를 입력으로 context vector `z`를 얻을 수 있어요!

$$z = EncoderRNN(e(x_T),h_{T-1})$$

- 이제 decoder에 어떤 값이 들어가는지 확인해볼까요?
- good morning을 출력하기 위해 문장 시작 토큰 `<sos>`과 context vector `z`를 Decoder RNN에 넣어줄 거에요

$$s_t = DecoderRNN(d(y_t),s_{t-1})$$

- `e`와 `d`는 보통 같은 Cell로 구현이 되서 같은 연산을 실시해요! (e.g., LSTM, GRU)
- 여기선 인코더와 디코더의 파라미터를 사용한다는 것을 강조하기 위해 표기를 구분지었어요
- 이게 굉장히 중요할 수 있는데요, 보통 huggingface에선 `lm_head`로 구현되어 있어요
- decoder의 hidden state로 어떻게 단어 토큰을 생성해낼까요?
- decoder output의 shape을 잘 생각해보면, `(batch_size, 1, hidden_dim)`일거에요
    - 각 time-step별!
- 이를 squeeze해주고 `hidden_dim` -> `vocab_size`로 보내주는 linear transformation을 취해주고 softmax를 통해 확률로 만들어주면?
- 해당 step에서 어떤 token이 나올지에 대한 확률 비스무리한 것이 나오게 될거에요!
    - softmax는 calibration을 하지 않는 이상, 확률이 아니에요 ㅎㅎ
- 학습 땐 정답을 decoder input에 넣어주는 teacher forcing을 할거고
- 보통 추론 때는 위 확률 분포에서 token을 추출하여 다음 decoder input에 넣어주는 input feeding을 실시해요
- 추론을 그렇다면 언제까지 하는가? max output length를 설정하거나 `<eos>` token이 나올 때까지 계속 뽑는답니다.
- 확률 분포에서 argmax로 token을 뽑을 수도 있고 greedy decoding이나 beam search, sampling 기법의 top-k, top-p, 혹은 Meena에서 사용한 softmax calibration 기법을 사용할 수도 있어요!

- 우리의 예측 문장은 $\hat{Y}=\{\hat{y_1},\hat{y_2},\cdots,\hat{y_T}\}$이 될거고 타켓 문장은 $Y=\{y_1,y_2,\cdots,y_T\}$이 될 거에요!
- 학습 땐 둘의 길이가 같겠죠? (왜냐면 teacher forcing!)
- cross entropy loss로 정답을 구할거구요
- 추론 땐 정답이 없어요! BLEU score 등을 구할 때 원하는 candidate set과 우리가 추론한 정답의 길이는 다를 수 있다는 것!
- 예를 들어,
    - x = "아버지가 방에 들어가신다"
    - golden_y = "아버지가 방에 들어가셨다는 사실을 알고있니?"
    - inference_y = "아버지가 밤에 도대체 어딜 들어가셨다는거야?"

## Preparing Data
- 자, 데이터를 준비해보자구요 ㅎㅎ

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data, datasets
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [26]:
import torchtext
print(torchtext.__version__)

0.5.0


In [27]:
torch.__version__

'1.7.1+cu101'

In [31]:
spacy.__version__

'2.2.4'

In [32]:
np.__version__

'1.19.5'

- Seed를 고정해줄거에요.
- 아래 코드는 기억해두시는게 좋아요! 아님 git gist에 적어두시거나

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

- 다음은 tokenizer를 만들겁니다.
- tokenizer는 말 그대로 분절하는 것을 의미해요!
- 아래와 같은 식으로 말이죠.
    - good morning! --> ["good", "morning", "!"]
- 즉, token들의 sequence죠?
- 여기서 `!`와 같이 중요하지 않을 수 있는 토큰들은 stop_words 처리해주는 것이 model performance 상으로 괜찮을 수 있어요.
- spacy는 위와 같은 작업을 해주지만, 현업에 가면 직접 사전을 구축해서 넣어주는 일이 필요할 수 있어요!
- 그러니까 언젠가는 사전 관련 내부 코드 등을 뜯어봐야 하는 날이 올 수도 있다는 얘기죠 ㅎㅎ

#### Note:
- 아래의 명령어로 코드를 받으세요!
```
python -m spacy download en
python -m spacy download de
```

In [3]:
import spacy

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

- 이제 tokenizer function을 만들어줄겁니다!
- 원래 source의 예시에는 source sentence를 뒤집어서 학습하는데
    - 논문의 구현에서, 이렇게 하면 최적화가 더 손쉽답니다
    - 그냥 backward pass로만 학습...
- 저는 그냥 진행했어요! 그래도 성능이 더 좋습니다

In [4]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

- `torchtext`의 `Field` 객체는 data를 어떻게 처리할 지 다룬다고 하네요
- 아래와 같이 argument를 넣어주면 다음과 같이 문장이 전처리됩니다.
    - Good Morning
    - good morning
    - ["good", "morning"]
    - ["`<sos>`", "good", "morning", |"`<eos>`"]

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

- 아래의 코드로 train, valid, test 데이터셋을 다운로드 받을 겁니다!
- 데이터 셋은 Multi30K라는 기계번역 데이터셋을 활용할 거구요
- 30K의 parallel En<->Ge, Fr로 되어있어요!
- 문장 당 단어는 최대 12 단어이구요
- `exts`는 source/target 언어를 지정해주는 인자에요

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

- 데이터를 올바르게 불러왔는지 확인해봅시다

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


- 원래 github source와는 다르게 반전안할거에요!
- 저는 그게 더 성능이 좋더라구요
- 휴리스틱은 음... 글쎄요 해보고 믿읍시다!

In [8]:
print(vars(train_data.examples[0]))

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


- 이제 vocab을 설정할건데요, 이것도 nlp에서 중요한 작업입니다
- 별로 나오지 않는 단어를 설정하면 lm_head의 차원만 커지니 비효율적이에요!
- 전략적으로 다가가야 합니다

In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [10]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


- torch device 설정

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- batch iterator 설정

In [12]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Building the Seq2Seq Model

### Encoder
- 2 layer LSTM
- input sentence $X$
- hidden state at layer i $H^i=\{h_1^i,h_2^i,\cdots,h_T^i\}$
- first layer hidden state each time steps

$$h_t^1=\text{EncoderRNN}^1(e(x_t),h_{t-1}^1)$$

- second layer hidden state each time steps

$$h_t^2=\text{EncoderRNN}^2(h_{t}^1,h_{t-1}^2)$$

- context vector each layers

$$z^1=h_T^1$$
$$z^2=h_T^2$$

- LSTM은 hidden state에 cell state도 있는거 기억하시죠?
- 어려운 수식 말고 한 번 직접 짜봅시다

---

### 헷갈리지만 중요하고 유용한 사실들!
- 본 source code에서 input의 모양은 `(sequence_length, batch_size, hidden_dim)`이에요
- 하지만 보통은 그렇게 넣어주지 않죠! `(batch_size, sequence_length, hidden_dim)`으로 넣어준답니다.
- 어떻게 설정할까요? 그리고 cell state...?
- input/output 어떻게 나오고 들어오는지 한 번 뜯어봅시다

**LSTM default values**

In [61]:
x = torch.randn(3, 5, 10) # (batch_size, seq_len, hidden_dim)

In [62]:
rnn = nn.LSTM(10, 5, batch_first=False, bidirectional=False, num_layers=1)
rnn

LSTM(10, 5)

In [64]:
output, (hidden, cell) = rnn(x.permute(1, 0, 2)) # permutation

In [66]:
output.size() # (seq_len, batch_size, hidden_dim)

torch.Size([5, 3, 5])

In [68]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)

(torch.Size([1, 3, 5]), torch.Size([1, 3, 5]))

**LSTM batch_first=True**

In [69]:
rnn = nn.LSTM(10, 5, batch_first=True, bidirectional=False, num_layers=1)
rnn

LSTM(10, 5, batch_first=True)

In [70]:
output, (hidden, cell) = rnn(x)

In [71]:
output.size() # (batch_size, seq_len, hidden_dim)

torch.Size([3, 5, 5])

In [72]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)

(torch.Size([1, 3, 5]), torch.Size([1, 3, 5]))

**LSTM batch_first=True, num_layers=2**

In [73]:
rnn = nn.LSTM(10, 5, batch_first=True, bidirectional=False, num_layers=2)
rnn

LSTM(10, 5, num_layers=2)

In [74]:
output, (hidden, cell) = rnn(x)

In [75]:
output.size() # (batch_size, seq_len, hidden_dim)
              # layer만 높인거라 size에 변화 X

torch.Size([3, 5, 5])

In [76]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)
                           # layer별 결과도 가져오기 때문에 size up

(torch.Size([2, 5, 5]), torch.Size([2, 5, 5]))

**LSTM batch_first=True, num_layers=2, bidirectional=True**

In [77]:
rnn = nn.LSTM(10, 5, batch_first=True, bidirectional=True, num_layers=2)
rnn

LSTM(10, 5, num_layers=2, bidirectional=True)

In [78]:
output, (hidden, cell) = rnn(x)

In [79]:
output.size() # (batch_size, seq_len, hidden_dim * 2)
              # forward and backward pass 둘을 concat해주기 때문

torch.Size([3, 5, 10])

In [80]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)

(torch.Size([4, 5, 5]), torch.Size([4, 5, 5]))

**dropout은 n_layer>=2부터**

In [82]:
nn.LSTM(10, 5, batch_first=True, num_layers=2, dropout=0.2)

LSTM(10, 5, num_layers=2, batch_first=True, dropout=0.2)

In [83]:
nn.LSTM(10, 5, batch_first=True, num_layers=1, dropout=0.2)

C:\Users\jinma\AppData\Local\Continuum\anaconda3\envs\basic\lib\site-packages\torch\nn\modules\rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


LSTM(10, 5, batch_first=True, dropout=0.2)

---

![image.png](https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013/assets/seq2seq2.png)

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

### Decoder

![image.png](https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013/assets/seq2seq3.png)

- lm_head 추가
- 단일 토큰별로 처리하도록 forward가 구성되어 있음.

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        # lm_head
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

### Seq2Seq
- 입력/소스 문장 받기
- 인코더를 활용하여 context vector 생성
- 디코더를 활용하여 예측 문장 / 목료 문장 생성

![image.png](https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013/assets/seq2seq4.png)

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

### Training the seq2seq model
- 모델 초기화
- 파라미터 설정

In [16]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

- 파라미터를 -0.08 ~ -0,08의 uniform 분포로 초기화시키자!
- `nn.init.uniform_`은 in-place 인자임

In [17]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

- 총 14M의 parameter를 가지고 있음

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,899,013 trainable parameters


- Adam optimizer 사용

In [19]:
optimizer = optim.Adam(model.parameters())

- Cross Entropy Loss 사용

#### CELoss는 어떻게 동작할까?

In [94]:
y_pred = torch.randn(10, 3, 30000) # (seq_len, bsz, vocab_size)

In [97]:
y_pred

tensor([[[-0.0828,  0.4125,  0.3340,  ...,  0.7334,  0.3160,  0.6537],
         [-2.4486,  0.1952, -0.2943,  ..., -1.9048,  1.5262, -1.1121],
         [-1.6414, -1.0252, -1.4527,  ...,  0.6101, -0.2816, -0.0311]],

        [[ 0.9988,  0.0054,  1.3491,  ...,  0.8649, -1.0560,  1.4901],
         [ 0.8748, -0.5448, -1.5865,  ...,  0.0784, -1.8257, -0.0904],
         [ 0.4253, -0.7102, -1.3635,  ..., -2.2506,  0.5756,  0.4709]],

        [[-0.0050, -0.1499,  0.2189,  ...,  0.5449, -0.9731, -0.0635],
         [ 1.2211, -1.7810, -0.0653,  ..., -0.0360, -1.1089, -0.4934],
         [ 0.5145, -0.7971,  0.7535,  ...,  0.1521, -0.0868,  0.1423]],

        ...,

        [[ 0.9344, -1.9056, -1.3603,  ..., -0.3813, -0.7312,  0.0287],
         [-0.7076,  1.0310,  1.7773,  ..., -0.6313, -0.7449, -0.9435],
         [-0.7282,  0.0181,  1.0175,  ..., -0.0327, -0.8533, -0.4392]],

        [[ 0.2228,  0.9212, -0.5597,  ...,  0.2300,  2.3570, -1.0053],
         [-0.3954, -0.9313, -1.0421,  ..., -0.6412,  0.

In [112]:
y = torch.LongTensor(10, 3).random_(1, 29999)
y[-3:, 0] = 0
y[-2:, 2] = 0
y

tensor([[19241,  2759, 10354],
        [14284,  4891, 17877],
        [17212, 21004, 19434],
        [20941,   732, 26679],
        [16302,    87, 17223],
        [29049,  4606, 14762],
        [ 7922, 21280, 12773],
        [    0, 16778, 25911],
        [    0, 26360,     0],
        [    0, 24974,     0]])

In [113]:
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [117]:
criterion(y_pred.view(-1, 30000), y.view(-1))

tensor(10.8252)

In [120]:
nn.NLLLoss(ignore_index=0)(
    torch.log_softmax(y_pred.view(-1, 30000), dim=-1), 
    y.view(-1)
)

tensor(10.8252)

---

In [20]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [121]:
TRG_PAD_IDX

1

각 iteration에서 아래의 과정을 수행합니다!

1. source / target 문장을 batch로 받습니다
2. model의 gradient를 0으로 조정해줘요!
3. 모델에 src/tgt를 넣어줘서 예측값을 받습니다
4. Cross Entropy로 loss를 계산합니다.
5. `loss.backward()` 메서드로 gradient를 계산합니다.
6. exploding gradient를 방지하기 위해 gradient를 clip해줍니다
7. optimizer class에 정의된 최적화 기법으로 파라미터를 update해줍니다.
8. total loss를 더해줍니다.

In [21]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

- `.eval()`을 해주면 model의 dropout이 동작하지 않아요!
- `torch.no_grad`를 통해 gradient를 계산하지 않도록 해줍니다.
    - context manager
- 추론에선 teacher forcing을 사용하면 안되요! 정답이 없기 때문이죠

In [22]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

- 학습에 소요된 시간을 체크해봅시다.

In [23]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

- best 경신 때마다 그 때의 parameter를 저장할거에요!
- Cross Entropy에 exponential을 취해주면 PPL이 나와요!

- https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-8/03-perpexity

In [24]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 35s
	Train Loss: 4.827 | Train PPL: 124.780
	 Val. Loss: 4.798 |  Val. PPL: 121.305
Epoch: 02 | Time: 0m 35s
	Train Loss: 4.147 | Train PPL:  63.252
	 Val. Loss: 4.362 |  Val. PPL:  78.403
Epoch: 03 | Time: 0m 35s
	Train Loss: 3.809 | Train PPL:  45.086
	 Val. Loss: 4.238 |  Val. PPL:  69.298
Epoch: 04 | Time: 0m 35s
	Train Loss: 3.607 | Train PPL:  36.860
	 Val. Loss: 4.024 |  Val. PPL:  55.916
Epoch: 05 | Time: 0m 35s
	Train Loss: 3.414 | Train PPL:  30.384
	 Val. Loss: 3.953 |  Val. PPL:  52.116
Epoch: 06 | Time: 0m 35s
	Train Loss: 3.247 | Train PPL:  25.725
	 Val. Loss: 3.880 |  Val. PPL:  48.437
Epoch: 07 | Time: 0m 35s
	Train Loss: 3.093 | Train PPL:  22.041
	 Val. Loss: 3.790 |  Val. PPL:  44.264
Epoch: 08 | Time: 0m 36s
	Train Loss: 2.938 | Train PPL:  18.882
	 Val. Loss: 3.725 |  Val. PPL:  41.452
Epoch: 09 | Time: 0m 36s
	Train Loss: 2.817 | Train PPL:  16.726
	 Val. Loss: 3.761 |  Val. PPL:  42.985
Epoch: 10 | Time: 0m 35s
	Train Loss: 2.698 | Train PPL

In [25]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.709 | Test PPL:  40.803 |
